In [ ]:
!pip install transformers==4.17
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [ ]:
splits = {'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

display(train_df.head())
display(test_df.head())
display(train_df.shape)
display(test_df.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,AccessMixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,AmbiguityError,more than one migration matches a name prefix.,1,more than one migration matches a name prefix....,"[0, 0, 0, 0, 1]"
2,3,AppConfigStub,stub of an appconfig.,1,stub of an appconfig. | AppConfigStub,"[0, 0, 0, 0, 1]"
3,4,AppConfigStub,only provides a label and a dict of models.,1,only provides a label and a dict of models. | ...,"[0, 1, 0, 0, 0]"
4,5,Archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


(1884, 6)

(406, 6)

In [ ]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,1,accessmixin,functionality.,0,functionality | accessmixin,"[0, 0, 0, 0, 1]"
1,8,atomic,"when it s used as a decorator, call wraps the ...",0,when it s used as a decorator call wraps the e...,"[1, 0, 0, 0, 0]"
2,10,atomic,"when it s used as a context manager, enter cre...",0,when it s used as a context manager enter crea...,"[1, 0, 0, 0, 0]"
3,12,atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,accessmixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,ambiguityerror,more than one migration matches a name prefix.,1,more than one migration matches a name prefix ...,"[0, 0, 0, 0, 1]"
2,3,appconfigstub,stub of an appconfig.,1,stub of an appconfig | appconfigstub,"[0, 0, 0, 0, 1]"
3,4,appconfigstub,only provides a label and a dict of models.,1,only provides a label and a dict of models | a...,"[0, 1, 0, 0, 0]"
4,5,archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [ ]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [ ]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best trial:")
print(study.best_trial)

[I 2024-11-30 19:27:37,409] A new study created in memory with name: no-name-cf2b126a-43ef-473f-8607-2a3a514fd55e
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.489946,0.783024,0.000000,0.000000,0.000000
1,No log,0.452661,0.810610,0.501702,0.141809,0.205888
2,No log,0.435868,0.828647,0.473789,0.256724,0.329053


***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-11-30 19:29:37,619] Trial 0 finished with value: 0.3290530233915995 and parameters: {'learning_rate': 9.936905260984444e-06, 'batch_size': 8, 'num_train_epochs': 3, 'weight_decay': 0.08999766298907733}. Best is trial 0 with value: 0.3290530233915995.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.341361,0.863660,0.705112,0.488998,0.560276
1,No log,0.317425,0.875862,0.692439,0.621027,0.652749
2,0.295500,0.319571,0.878515,0.755899,0.657702,0.682680
3,0.295500,0.361003,0.879576,0.752510,0.682152,0.691598
4,0.295500,0.334092,0.897613,0.784268,0.728606,0.752711
5,0.103400,0.333054,0.901857,0.802773,0.733496,0.764948
6,0.103400,0.340389,0.900265,0.794929,0.723716,0.757056
7,0.036200,0.350222,0.898143,0.781876,0.728606,0.752393


...

[I 2024-11-30 20:48:26,144] Trial 18 finished with value: 0.7571633066830137 and parameters: {'learning_rate': 8.06494012495203e-06, 'batch_size': 16, 'num_train_epochs': 7, 'weight_decay': 0.0037009462787351926}. Best is trial 14 with value: 0.7664093810806344.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning:

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.844785,0.896021,0.767298,0.748166,0.756031
1,No log,0.862137,0.897082,0.770002,0.757946,0.761264
2,No log,0.860023,0.896021,0.767753,0.753056,0.756863
3,No log,0.857765,0.897082,0.769532,0.755501,0.759864
4,No log,0.851376,0.896021,0.765364,0.753056,0.757585
5,0.000000,0.863781,0.896552,0.771111,0.753056,0.758970
6,0.000000,0.863133,0.897613,0.775229,0.753056,0.760727
7,0.000000,0.862491,0.897082,0.772598,0.753056,0.759739
8,0.000000,0.862999,0.897613,0.774384,0.755501,0.761396


***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 377
  Batch size = 16


[I 2024-11-30 20:54:05,984] Trial 19 finished with value: 0.7613955355470121 and parameters: {'learning_rate': 1.945827508559902e-06, 'batch_size': 8, 'num_train_epochs': 9, 'weight_decay': 0.02037909320490351}. Best is trial 14 with value: 0.7664093810806344.


Best trial:
FrozenTrial(number=14, state=TrialState.COMPLETE, values=[0.7664093810806344], datetime_start=datetime.datetime(2024, 11, 30, 20, 29, 18, 475437), datetime_complete=datetime.datetime(2024, 11, 30, 20, 32, 47, 656203), params={'learning_rate': 2.0071035169373224e-05, 'batch_size': 16, 'num_train_epochs': 6, 'weight_decay': 0.028779217423793482}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=14, value=None)


In [ ]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 282


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.923222,0.894960,0.768639,0.748166,0.753845
1,No log,0.971287,0.891247,0.761478,0.740831,0.743622
2,No log,0.908051,0.893899,0.766098,0.745721,0.753367
3,No log,0.915926,0.896021,0.768218,0.753056,0.757172
4,No log,0.915880,0.894430,0.764844,0.748166,0.753034
5,No log,0.915147,0.894960,0.767088,0.748166,0.754075


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=282, training_loss=9.004434848085362e-05, metrics={'train_runtime': 206.0722, 'train_samples_per_second': 43.878, 'train_steps_per_second': 1.368, 'total_flos': 598718438392320.0, 'train_loss': 9.004434848085362e-05, 'epoch': 5.99})

In [ ]:
!mkdir 'best_model_distilbert_python'
!mkdir 'best_model_distilbert_python_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_distilbert_python')
tokenizer.save_pretrained('./best_model_distilbert_python_tokenizer')

# Load model and tokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('./best_model_distilbert_python')
tokenizer = DistilBertTokenizer.from_pretrained('./best_model_distilbert_python_tokenizer')

Saving model checkpoint to ./best_model_distilbert_python
Configuration saved in ./best_model_distilbert_python/config.json
Model weights saved in ./best_model_distilbert_python/pytorch_model.bin
tokenizer config file saved in ./best_model_distilbert_python_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_distilbert_python_tokenizer/special_tokens_map.json
loading configuration file ./best_model_distilbert_python/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
 

## Model Testing

In [ ]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [ ]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

print(test_label)

[array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 1, 0, 0, 0]) array([0, 0, 1, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([1, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 1, 0, 0])
 array([0, 0, 0, 0, 

In [ ]:
type(test_label[0])

numpy.ndarray

In [ ]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 406
  Batch size = 32


Evaluation Metrics: {'accuracy': 0.8605911330049261, 'precision': 0.6843921427560762, 'recall': 0.6422018348623854, 'f1': 0.6591660558686011}
Inference Time: 3.31 seconds


In [ ]:
!zip -r best_model_distilbert_python.zip './best_model_distilbert_python'
!zip -r best_model_distilbert_python_tokenizer.zip './best_model_distilbert_python_tokenizer'

  adding: best_model_distilbert_python/ (stored 0%)
  adding: best_model_distilbert_python/pytorch_model.bin (deflated 8%)
  adding: best_model_distilbert_python/training_args.bin (deflated 51%)
  adding: best_model_distilbert_python/config.json (deflated 51%)
  adding: best_model_distilbert_python_tokenizer/ (stored 0%)
  adding: best_model_distilbert_python_tokenizer/tokenizer_config.json (deflated 43%)
  adding: best_model_distilbert_python_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_distilbert_python_tokenizer/vocab.txt (deflated 53%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_distilbert_python.zip "/content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python/"
!cp best_model_distilbert_python_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python_Tokenizer/"

Mounted at /content/drive


# **Load and Test Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model_folder = '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python/best_model_distilbert_python.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python_Tokenizer/best_model_distilbert_python_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python/best_model_distilbert_python.zip
replace ./best_model_distilbert_python/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_distilbert_python/pytorch_model.bin  
  inflating: ./best_model_distilbert_python/training_args.bin  
  inflating: ./best_model_distilbert_python/config.json  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python_Tokenizer/best_model_distilbert_python_tokenizer.zip
replace ./best_model_distilbert_python_tokenizer/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_distilbert_python_tokenizer/tokenizer_config.json  
  inflating: ./best_model_distilbert_python_tokenizer/special_tokens_map.json  
  inflating: ./best_model_distilbert_python_tokenizer/vocab.txt  


In [ ]:
best_model_distilbert_python = 'best_model_distilbert_python'
best_model_distilbert_python_tokenizer = 'best_model_distilbert_python_tokenizer'

In [ ]:
# Load the model and tokenizer from the checkpoint
model = DistilBertForSequenceClassification.from_pretrained(best_model_distilbert_python)
tokenizer = DistilBertTokenizer.from_pretrained(best_model_distilbert_python_tokenizer)

text = "what is this?"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

loading configuration file best_model_distilbert_python/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading weights file best_model_distilbert_python/pytorch

SequenceClassifierOutput(loss=None, logits=tensor([[-10.4652, -11.5933,   6.8568,  -9.4027, -12.2137]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 3
